# This example demonstrate how to train YOLO detector model

In [1]:
import sys
import os
import json
import yaml
import shutil
import matplotlib.pyplot as plt

# change this property
NOMEROFF_NET_DIR = os.path.abspath('../')
sys.path.append(NOMEROFF_NET_DIR)

from NomeroffNet.tools.yolo_tools import convert_dataset_to_yolo_format

In [2]:
plt.rcParams["figure.figsize"] = (20, 10)

In [3]:
%matplotlib inline 

# Download dataset and yolo repo

In [4]:
# auto download latest dataset
from NomeroffNet.tools import modelhub

# get yolov5
info = modelhub.download_repo_for_model("yolov5")
repo_path = info["repo_path"]

# auto download latest dataset
info = modelhub.download_dataset_for_model("yolov5")
PATH_TO_DATASET = info["dataset_path"]

# Convert dataset to yolo format

In [5]:
CLASSES = ['numberplate']
STATES  = ["val","train"]
#STATES  = ["val"]

PATH_TO_RES_ANN = os.path.join(os.path.dirname(PATH_TO_DATASET), "./npdata/labels/{}")
PATH_TO_RES_IMAGES  = os.path.join(os.path.dirname(PATH_TO_DATASET), "./npdata/images/{}")

PATH_TO_JSON = os.path.join(PATH_TO_DATASET, "{}/via_region_data.json")
PATH_TO_IMAGES = os.path.join(PATH_TO_DATASET, "{}/")

In [6]:
EXIST_STRATEGY = "exist_ok" # "delete", "exist_ok"

for state in STATES:
    path_to_res_ann    = PATH_TO_RES_ANN.format(state)
    path_to_res_images = PATH_TO_RES_IMAGES.format(state)
    
    if os.path.exists(path_to_res_ann) and os.path.exists(path_to_res_images) and EXIST_STRATEGY == "exist_ok":
        print("[INFO]", state, "data exists")
        continue
    if EXIST_STRATEGY == "delete":
        shutil.rmtree(path_to_res_ann)
        shutil.rmtree(path_to_res_images)
    
    print("[INFO]", state, "data creating...")
    os.makedirs(path_to_res_ann, exist_ok=True)
    os.makedirs(path_to_res_images, exist_ok=True)
    
    path_to_json    = PATH_TO_JSON.format(state)
    path_to_images  = PATH_TO_IMAGES.format(state)
    
    convert_dataset_to_yolo_format(
        path_to_res_ann, 
        path_to_res_images,
        path_to_images, 
        path_to_json, 
        debug=False,
        is_generate_image_rotation_variants=True
    )
    

[INFO] val data exists
[INFO] train data exists


# Create yaml config

In [7]:
yaml_path = os.path.join(os.path.dirname(PATH_TO_DATASET), "./npdata/numberplate_config.yaml")
yaml_dataset_config = {
    # Train/val/test sets
    "path": os.path.dirname(PATH_TO_DATASET), 
    "train": "./npdata/images/train",
    "val": "./npdata/images/val",
    #"test": "" # test images (optional)
    
    # Classes
    "nc": 1,  # number of classes
    "names": [ "numberplate" ]  # class names
}
print("[INFO] config:", yaml_dataset_config)
print("[INFO] config saved into", yaml_path)
with open(yaml_path, 'w') as yaml_file:
    yaml.dump(yaml_dataset_config, 
              yaml_file, 
              default_flow_style=False)

[INFO] config: {'path': '/mnt/store/nomeroff-net/nomeroff-net/NomeroffNet/tools/../../data/./dataset/Detector/yolov5', 'train': './npdata/images/train', 'val': './npdata/images/val', 'nc': 1, 'names': ['numberplate']}
[INFO] config saved into /mnt/store/nomeroff-net/nomeroff-net/NomeroffNet/tools/../../data/./dataset/Detector/yolov5/./npdata/numberplate_config.yaml


# CLI yolo training

In [ ]:
!cd {repo_path}; \
 wandb offline; \
 python3 train.py \
    --img 640 \
    --batch 8 \
    --epochs 100 \
    --data {yaml_path} \
    --weights yolov5s.pt

W&B offline, running your script from this directory will only write metadata locally.
train: weights=yolov5s.pt, cfg=, data=/mnt/store/nomeroff-net/nomeroff-net/NomeroffNet/tools/../../data/./dataset/Detector/yolov5/./npdata/numberplate_config.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=100, batch_size=8, img_size=[640], rect=False, resume=False, nosave=False, notest=False, noautoanchor=False, evolve=None, bucket=, cache_images=False, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, entity=None, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias=latest, local_rank=-1
github: skipping check (Docker image), for updates see https://github.com/ultralytics/yolov5
YOLOv5 🚀 v5.0-276-gdabad57 torch 1.9.0+cu111 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11019.4375MB)

hyperparameters: lr0=0.01, lrf=0.2, momentum=0.937, weight_deca

     21/99     2.41G   0.01657  0.004783         0   0.02135        16       640: 100%|██████████| 3833/3833 [08:07<00:00,  7.86it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100%|██████████| 94/94 [00:17<00:00,  5.27it/s]
                 all       1504       1620      0.978       0.97       0.99      0.792

     Epoch   gpu_mem       box       obj       cls     total    labels  img_size
     22/99     2.41G    0.0164   0.00475         0   0.02115        15       640: 100%|██████████| 3833/3833 [08:08<00:00,  7.84it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100%|██████████| 94/94 [00:20<00:00,  4.67it/s]
                 all       1504       1620      0.982      0.966      0.989      0.794

     Epoch   gpu_mem       box       obj       cls     total    labels  img_size
     23/99     2.41G   0.01636  0.004735         0    0.0211        21       640: 100%|██████████| 3833/3833 [08:06<00:00,


     Epoch   gpu_mem       box       obj       cls     total    labels  img_size
     40/99     2.41G   0.01506  0.004377         0   0.01943        12       640: 100%|██████████| 3833/3833 [12:29<00:00,  5.11it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100%|██████████| 94/94 [00:15<00:00,  6.00it/s]
                 all       1504       1620      0.982       0.97      0.992      0.797

     Epoch   gpu_mem       box       obj       cls     total    labels  img_size
     41/99     2.41G   0.01497  0.004381         0   0.01935        13       640: 100%|██████████| 3833/3833 [12:32<00:00,  5.10it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100%|██████████| 94/94 [00:15<00:00,  6.01it/s]
                 all       1504       1620      0.982       0.97      0.992      0.797

     Epoch   gpu_mem       box       obj       cls     total    labels  img_size
     42/99     2.41G   0.01499  0.004351

In [9]:
repo_path

'/mnt/store/nomeroff-net/nomeroff-net/NomeroffNet/tools/../../data/./repos/Detector/yolov5'

## Run wandb localy

```
docker run --rm -it -v wandb:{repo_path}/wandb -p 8080:8080 --ip 0.0.0.0 --name wandb-local wandb/local
```